In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

In [8]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')


#工事費用
cost_df = DataFrame(pd.read_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/prefecture_cost.csv', encoding='shift_jis'))
pref = ['北海道',
        '青森',
        '岩手',
        '宮城',
        '秋田',
        '山形',
        '福島',
        '茨城',
        '栃木',
        '群馬',
        '埼玉',
        '千葉',
        '東京都',
        '神奈川',
        '新潟',
        '富山',
        '石川',
        '福井',
        '山梨',
        '長野',
        '岐阜',
        '静岡',
        '愛知',
        '三重',
        '滋賀',
        '京都',
        '大阪',
        '兵庫',
        '奈良',
        '和歌山',
        '鳥取',
        '島根',
        '岡山',
        '広島',
        '山口',
        '徳島',
        '香川',
        '愛媛',
        '高知',
        '福岡',
        '佐賀',
        '長崎',
        '熊本',
        '大分',
        '宮崎',
        '鹿児島',
        '沖繩']
pref_df = DataFrame(pref, columns=['prefecture'])
cost_df = pd.merge(pref_df, cost_df, on='prefecture')
#損害額の計算
def dmg_value(cost, total_area, dpc_rate, age, dmg_rate):
        #建物の金額
        house_price = cost * total_area
        #償却された金額
        dpc_price = house_price * 0.9 * dpc_rate * age
        #時価総額
        tmp_price = house_price - dpc_price
        #築年が多いため時価総額がマイナスになった場合の条件分岐
        if (tmp_price / house_price) < 0.05:
                tmp_price = house_price * 0.05
        #損害額
        dmg = tmp_price * dmg_rate
        return dmg

def house_price(cost, total_area):
        #減価償却なしの住宅の金額
        house_price = cost * total_area
        return house_price

%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
n = int(start_pref)
while n <= int(finish_pref):
    damage_path = f'./data/損害額data/{n}_step8_1.csv'
    df = pd.read_csv(damage_path)
    dmg_price_list = []

    #工事費用
    df['cost'] = np.where(df['estimated_structure'] == 0, cost_df['wood'][n] * 1000, cost_df['steel'][n] * 1000)
    #償却率
    df['dpc_rate'] = np.where(df['estimated_structure'] == 0, 0.031, 0.036)
    #築年数
    df['age'] = df['estimated_age'].map({1: 5, 2: 8, 3: 15.5, 4: 25.5, 5: 35.5, 6: 45.5, 7: 50})
    
    df['dmg_rate'] = np.where((df['A31_001'] == 11) | (df['A31_001'] == 21), np.where(df['floor'] == 0, 0.4, 0.35),
                          np.where((df['A31_001'] == 12) | (df['A31_001'] == 22), np.where(df['floor'] == 0, 0.6, 0.45),
                          np.where((df['A31_001'] == 13) | (df['A31_001'] == 23), np.where(df['floor'] == 0, 0.75, 0.5),
                          np.where(((df['A31_001'].between(14, 15)) | (df['A31_001'] >= 24)), np.where(df['floor'] == 0, 0.8, 0.55), 0.0))))
    
    #減価償却なしの住宅の価値
    df['house_price'] = df.apply(lambda row: house_price(row['cost'], row['total_area']), axis=1)
    
    #損害額の計算
    df['damage_price'] = df.apply(lambda row: dmg_value(row['cost'], row['total_area'], row['dpc_rate'], row['age'], row['dmg_rate']), axis=1)
    
    
    damage_output_path = f'./dev/損害額/{n}_step8_1_output.csv'
    df.to_csv(damage_output_path, index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コー : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [5]:
df.groupby('KEY_CODE', as_index=False)['damage_price'].sum()

,KEY_CODE,damage_price
0,141010180,9.604426e+08
1,141010490,2.556908e+09
2,141010560,1.623599e+09
3,141010570,1.237905e+09
4,141010580,1.910518e+09
...,...,...
1360,14366009001,2.109556e+08
1361,14366009002,5.511168e+05
1362,14366009003,9.168066e+07
1363,14366009004,9.471514e+07


In [13]:
df['KEY_CODE'].value_counts().sum()

265446

In [9]:
df

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
0,1573,133.797213,34.672247,0,166.805,1364,166.805,331012270,1,4,12,256000,0.036,25.5,0.60,42702080.0,4.452973e+06
1,1574,133.797034,34.671509,0,131.640,1364,131.640,331012270,1,2,12,256000,0.036,8.0,0.60,33699840.0,1.497890e+07
2,1695,133.797357,34.672400,0,96.400,1364,96.400,331012270,1,3,12,256000,0.036,15.5,0.60,24678400.0,7.370945e+06
3,1688,133.797788,34.673460,0,184.710,1364,184.710,331012270,1,3,11,256000,0.036,15.5,0.40,47285760.0,9.415541e+06
4,1694,133.797064,34.672907,0,102.940,1364,102.940,331012270,1,1,11,256000,0.036,5.0,0.40,26352640.0,8.833405e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333097,19877,133.817980,34.858477,0,253.700,1364,253.700,33681059005,1,6,12,256000,0.036,45.5,0.60,64947200.0,1.948416e+06
333098,19890,133.816148,34.860425,0,107.045,1364,107.045,33681059005,1,3,11,256000,0.036,15.5,0.40,27403520.0,5.456589e+06
333099,19898,133.817725,34.860710,0,118.770,1364,118.770,33681059006,1,1,11,256000,0.036,5.0,0.40,30405120.0,1.019180e+07
333100,19919,133.818189,34.857995,0,162.060,1364,162.060,33681059005,1,4,13,256000,0.036,25.5,0.75,41487360.0,5.407877e+06
